In [1]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/DataFusionContest/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.ensemble import RandomForestClassifier
#from matplotlib import pyplot
from sklearn.metrics import accuracy_score
import io
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import pickle

In [3]:
df = pd.read_csv('clean.csv')

In [4]:
# Получение списка уникальных значений категорий товара для кодирования методом
# One Hot Encoding
cols = list(df['category_id'].unique())
cols.sort()
print(cols)
dummiesColumnsName = []
for i in range(len(cols)):
    dummiesColumnsName.append(str(cols[i]))
print(dummiesColumnsName)

[0, 1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 19, 20, 24, 26, 27, 29, 30, 31, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 90, 92, 96, 97, 100, 101, 102, 103, 105, 106, 107, 108, 109, 111, 114, 115, 117, 118, 120, 121, 128, 130, 133, 138, 139, 140, 143, 145, 150, 163, 164, 167, 177, 203, 204]
['0', '1', '2', '3', '4', '6', '7', '9', '11', '12', '13', '19', '20', '24', '26', '27', '29', '30', '31', '35', '36', '37', '38', '39', '40', '41', '42', '43', '45', '46', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '60', '61', '62', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '90', '92', '96', '97', '100', '101', '102', '103', '105', '106', '107', '108', '109', '111', '114', '115', '117', '118', '120', '121', '128', '130', '133', '138', '139', '140', '143', '145', '150', '163', '164', 

In [5]:
# Создание датасета для обучения (первые 100 000 записей)
df2 = df.head(100000)
df2['item_name'] = df2['item_name'].str.lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# Использование токенайзера для преобразования названия товара (item_name)
# Фильтры:
to_exclude = '".,:!?#$%&()*+-/<=>@[\\]^_`{|}~\t\n' # Исключаемые символы
to_tokenize = '.,:!?"' # Обрабатываемые символы

# Обучение токенайзера
num_words = 20000 # Максимальное количество слов в словаре
max_review_len = 100 # Максимальное количество слов в названии товара
tokenizer_goods = Tokenizer(num_words=num_words, filters=to_exclude, oov_token=1)
tokenizer_goods.fit_on_texts(df2['item_name'])

print('Обучение завершено')

# Сохранение полученного словаря
tokenizer_json = tokenizer_goods.to_json()
with io.open('tokenizer_item_name.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

print('Результат успешно сохранен')

# Загрузка словаря токенайзера
with open('tokenizer_item_name.json') as f:
    data = json.load(f)
    tokenizer_goods = tokenizer_from_json(data)

print('Словарь токенайзер загружен')

# Обработка по полученному словарю
sequences_goods = tokenizer_goods.texts_to_sequences(df2['item_name'])
max_position_len=10
xTrain = pad_sequences(sequences_goods, maxlen=max_position_len)
print(xTrain[0])

Обучение завершено
Результат успешно сохранен
Словарь токенайзер загружен
[ 0  0  0  0  0  0 21 18  8  9]


In [7]:
yTrain = pd.DataFrame(np.zeros((len(df2), len(cols)), dtype=np.uint8), columns=dummiesColumnsName)
for i in range(len(df2)):
    j = df2.loc[i, 'category_id']
    yTrain.loc[i, str(j)] = 1
print(df2.loc[0, 'category_id'])
print(yTrain.loc[0, '78'])

78
1


In [12]:
# Обучение модели (Случайный Лес из 50 деревьев)
model = RandomForestClassifier(n_estimators=50)
model.fit(xTrain, yTrain)
# Вывод значимости параметров
importance = model.feature_importances_
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

#print('1')
#yhat = model.predict(x)
#print(y)
#print(yhat)

Feature: 0, Score: 0.00430
Feature: 1, Score: 0.00748
Feature: 2, Score: 0.01356
Feature: 3, Score: 0.02514
Feature: 4, Score: 0.04587
Feature: 5, Score: 0.06899
Feature: 6, Score: 0.11559
Feature: 7, Score: 0.17460
Feature: 8, Score: 0.21888
Feature: 9, Score: 0.32561


In [14]:
del df
del df2
del tokenizer_goods
del sequences_goods

In [ ]:
print(model.estimators_)
#print(treeObj.node_count)

In [15]:
# Сохранение модели
pickle.dump(model, open('model_v1.sav', 'wb'))

In [16]:
# Загрузка модели
model = pickle.load(open('model_v1.sav', 'rb'))

In [19]:
z_real = pd.DataFrame(pd.get_dummies(yTrain).idxmax(1), columns=['category_id'])
print(z_real.head(10))
y = model.predict(xTrain)
yPred = pd.DataFrame(y, columns=dummiesColumnsName)
z_pred = pd.DataFrame(pd.get_dummies(yPred).idxmax(1), columns=['category_id'])
z_pred['category_id'] = z_pred['category_id'].astype('uint8')
print(z_pred.head(10))

  category_id
0          78
1          71
2          71
3          70
4          84
5          84
6          69
7          69
8          69
9          68
   category_id
0           78
1           71
2           71
3           70
4           84
5           84
6           69
7           69
8           69
9           68


In [21]:
z_real['category_id'] = z_real['category_id'].astype('uint8')
print(accuracy_score(z_pred, z_real))

0.99394


При 100 деревьях было 0.99492

Дальше вниз надо чистить

Проверка на тренировочных данных (последние 1000 строк "очищенного" датасета)

In [15]:
#print(z.head(10))
cols = list(z['category_id'].unique())
cols.sort()
print(cols)
dummiesColumnsName = []
for i in range(len(cols)):
    dummiesColumnsName.append(str(cols[i]))
print(dummiesColumnsName)

y = pd.DataFrame(np.zeros((1000, len(cols)), dtype=np.uint8), columns=dummiesColumnsName)
print(y.head(10))

[0, 1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 19, 20, 24, 26, 27, 29, 30, 31, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 90, 92, 96, 97, 100, 101, 102, 103, 105, 106, 107, 108, 109, 111, 114, 115, 117, 118, 120, 128, 130, 133, 138, 139, 140, 143, 145, 150, 163, 164, 167, 177, 203, 204]
['0', '1', '2', '3', '4', '6', '7', '9', '11', '12', '13', '19', '20', '24', '26', '27', '29', '30', '31', '35', '36', '37', '38', '39', '40', '41', '42', '43', '45', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '60', '61', '62', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '90', '92', '96', '97', '100', '101', '102', '103', '105', '106', '107', '108', '109', '111', '114', '115', '117', '118', '120', '128', '130', '133', '138', '139', '140', '143', '145', '150', '163', '164', '167', '177', '203', '

In [16]:
df3 = df.tail(1000).reset_index()
df3['item_name'] = df3['item_name'].str.lower()
z_real = df3['category_id']

sequences_position = tokenizer_position.texts_to_sequences(df3['item_name'])
max_position_len=10
x = pad_sequences(sequences_position, maxlen=max_position_len)

for i in range(len(df3)):
    j = df3.loc[i, 'category_id']
    y.loc[i, str(j)] = 1
print(y.loc[0, '69'])
print(y.head(10))
print(z_real.head(10))

1
   0  1  2  3  4  6  7  9  11  ...  143  145  150  163  164  167  177  203  204
0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
1  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
2  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
3  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
4  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
5  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
6  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
7  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
8  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0
9  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0    0

[10 rows x 96 columns]
0     69
1     84
2     66
3     75
4     76
5    130
6     84
7     71
8     77
9     76
Name

Предсказание

In [17]:
yhat = model.predict(x)

In [21]:
y_pred = pd.DataFrame(yhat, columns=dummiesColumnsName)
z_pred = pd.DataFrame(pd.get_dummies(y_pred).idxmax(1), columns=['category_id'])
z_pred['category_id'] = z_pred['category_id'].astype('uint8')
print(z_real.head(10))
print(z_pred.head(10))

print(z_real.tail(10))
print(z_pred.tail(10))
print(accuracy_score(z_pred, z_real))

0     69
1     84
2     66
3     75
4     76
5    130
6     84
7     71
8     77
9     76
Name: category_id, dtype: int64
   category_id
0           69
1           84
2           66
3           75
4           76
5          130
6            0
7           71
8           77
9           76
990     78
991     78
992     72
993     83
994     78
995     73
996    203
997    203
998     71
999     84
Name: category_id, dtype: int64
     category_id
990           78
991           78
992           72
993           83
994           78
995            0
996          203
997          203
998           71
999           84
0.937


In [22]:
print(accuracy_score(z_pred, z_real))

0.937
